In [1]:
from tqdm import tqdm_notebook as tqdm

In [2]:
type(tqdm)

function

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
structures = pd.read_csv('//Users/simonholmes/molecular_properties/runModel/run/data/structures.csv')

In [5]:
electronegativity = {'H':2.2, 'C':2.55, 'N':3.04, 'O':3.44, 'F':3.98}

In [6]:
atomic_radius = {'H':0.38, 'C':0.77, 'N':0.75, 'O':0.73, 'F':0.71}

In [7]:
atoms = structures['atom'].values
atoms_en = [electronegativity[x] for x in tqdm(atoms)]
atoms_rad = [atomic_radius[x] for x in tqdm(atoms)]

In [8]:
structures['EN'] = atoms_en
structures['rad'] = atoms_rad

display(structures.head())

,molecule_name,atom_index,atom,x,y,z,EN,rad
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001,2.55,0.77
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976,2.20,0.38
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277,2.20,0.38
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644,2.20,0.38
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397,2.20,0.38


In [9]:
df_test = pd.read_csv('//Users/simonholmes/molecular_properties/runModel/run/data/df_test.csv')

In [10]:
i_atom = structures['atom_index'].values
p = structures[['x', 'y', 'z']].values
p_compare = p
m = structures['molecule_name'].values
m_compare = m
r = structures['rad'].values
r_compare = r

source_row = np.arange(len(structures))
max_atoms = 28

bonds = np.zeros((len(structures)+1, max_atoms+1), dtype=np.int8)
bond_dists = np.zeros((len(structures)+1, max_atoms+1), dtype=np.float32)

print('Calculating bonds')

for i in tqdm(range(max_atoms-1)):
    p_compare = np.roll(p_compare, -1, axis=0)
    m_compare = np.roll(m_compare, -1, axis=0)
    r_compare = np.roll(r_compare, -1, axis=0)
    
    mask = np.where(m == m_compare, 1, 0) #Are we still comparing atoms in the same molecule?
    dists = np.linalg.norm(p - p_compare, axis=1) * mask
    r_bond = r + r_compare
    
    bond = np.where(np.logical_and(dists > 0.0001, dists < r_bond), 1, 0)
    
    source_row = source_row
    target_row = source_row + i + 1 #Note: Will be out of bounds of bonds array for some values of i
    target_row = np.where(np.logical_or(target_row > len(structures), mask==0), len(structures), target_row) #If invalid target, write to dummy row
    
    source_atom = i_atom
    target_atom = i_atom + i + 1 #Note: Will be out of bounds of bonds array for some values of i
    target_atom = np.where(np.logical_or(target_atom > max_atoms, mask==0), max_atoms, target_atom) #If invalid target, write to dummy col
    
    bonds[(source_row, target_atom)] = bond
    bonds[(target_row, source_atom)] = bond
    bond_dists[(source_row, target_atom)] = dists
    bond_dists[(target_row, source_atom)] = dists

bonds = np.delete(bonds, axis=0, obj=-1) #Delete dummy row
bonds = np.delete(bonds, axis=1, obj=-1) #Delete dummy col
bond_dists = np.delete(bond_dists, axis=0, obj=-1) #Delete dummy row
bond_dists = np.delete(bond_dists, axis=1, obj=-1) #Delete dummy col

print('Counting and condensing bonds')

bonds_numeric = [[i for i,x in enumerate(row) if x] for row in tqdm(bonds)]
bond_lengths = [[dist for i,dist in enumerate(row) if i in bonds_numeric[j]] for j,row in enumerate(tqdm(bond_dists))]
bond_lengths_mean = [ np.mean(x) for x in bond_lengths]
bond_lengths_std = [ np.std(x) for x in bond_lengths]
n_bonds = [len(x) for x in bonds_numeric]

#bond_data = {'bond_' + str(i):col for i, col in enumerate(np.transpose(bonds))}
#bond_data.update({'bonds_numeric':bonds_numeric, 'n_bonds':n_bonds})

bond_data = {'n_bonds':n_bonds, 'bond_lengths_mean': bond_lengths_mean,'bond_lengths_std':bond_lengths_std }
bond_df = pd.DataFrame(bond_data)
structures = structures.join(bond_df)
display(structures.head(20))

Calculating bonds



Counting and condensing bonds


/Users/simonholmes/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/simonholmes/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/simonholmes/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/Users/simonholmes/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/simonholmes/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,molecule_name,atom_index,atom,x,y,z,EN,rad,n_bonds,bond_lengths_mean,bond_lengths_std
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001,2.55,0.77,4,1.091950,0.000003
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976,2.20,0.38,1,1.091953,0.000000
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277,2.20,0.38,1,1.091952,0.000000
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644,2.20,0.38,1,1.091946,0.000000
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397,2.20,0.38,1,1.091948,0.000000
5,dsgdb9nsd_000002,0,N,-0.040426,1.024108,0.062564,3.04,0.75,3,1.017195,0.000009
6,dsgdb9nsd_000002,1,H,0.017257,0.012545,-0.027377,2.20,0.38,1,1.017190,0.000000
7,dsgdb9nsd_000002,2,H,0.915789,1.358745,-0.028758,2.20,0.38,1,1.017187,0.000000
8,dsgdb9nsd_000002,3,H,-0.520278,1.343532,-0.775543,2.20,0.38,1,1.017208,0.000000
9,dsgdb9nsd_000003,0,O,-0.034360,0.977540,0.007602,3.44,0.73,2,0.962107,0.000000


In [13]:
def map_atom_info(df_1,df_2, atom_idx):
    print('Mapping...', df_1.shape, df_2.shape, atom_idx)

    df = pd.merge(df_1, df_2.drop_duplicates(subset=['molecule_name', 'atom_index']), how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])

    df = df.drop('atom_index', axis=1)
    
    return df


for atom_idx in [0,1]:
    df_test = map_atom_info(df_test,structures, atom_idx)
    df_test = df_test.rename(columns={'atom': f'atom_{atom_idx}',
                                    'x': f'x_{atom_idx}',
                                    'y': f'y_{atom_idx}',
                                    'z': f'z_{atom_idx}'})
#        df_test = map_atom_info(df_test,structures, atom_idx)
#        df_test = df_test.rename(columns={'atom': f'atom_{atom_idx}',
#                                'x': f'x_{atom_idx}',
#                                'y': f'y_{atom_idx}',
#                                'z': f'z_{atom_idx}'})

Mapping... (2505542, 69) (2358657, 11) 0
Mapping... (2505542, 78) (2358657, 11) 1


In [15]:
df_test.dtypes

id                        int64
molecule_name            object
atom_index_0              int64
atom_index_1              int64
type                     object
atom_0                   object
x_0                     float64
y_0                     float64
z_0                     float64
atom_1                   object
x_1                     float64
y_1                     float64
z_1                     float64
c_x                     float64
c_y                     float64
c_z                     float64
atom_n                    int64
dx                      float64
dy                      float64
dz                      float64
distance                float64
atom_index_closest_0      int64
distance_closest_0      float64
x_closest_0             float64
y_closest_0             float64
z_closest_0             float64
max_distance_x          float64
atom_index_closest_1      int64
distance_closest_1      float64
x_closest_1             float64
                         ...   
cos_f1  

In [18]:
df_test.to_csv('//Users/simonholmes/molecular_properties/runModel/run2/data/df_test_2.csv', index=False)

In [16]:
df_test.shape

(2505542, 87)

In [17]:
df_test.head()

,id,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,...,bond_lengths_std_y,atom_1,x_1,y_1,z_1,EN,rad,n_bonds,bond_lengths_mean,bond_lengths_std
0,4658147,dsgdb9nsd_000004,2,0,2JHC,H,-1.661639,0.0,1.0,C,...,0.0,C,0.599539,0.0,1.0,2.55,0.77,2,1.130589,0.06849
1,4658148,dsgdb9nsd_000004,2,1,1JHC,H,-1.661639,0.0,1.0,C,...,0.0,C,-0.599539,0.0,1.0,2.55,0.77,2,1.130589,0.06849
2,4658149,dsgdb9nsd_000004,2,3,3JHH,H,-1.661639,0.0,1.0,H,...,0.0,H,1.661639,0.0,1.0,2.20,0.38,1,1.062099,0.00000
3,4658150,dsgdb9nsd_000004,3,0,1JHC,H,1.661639,0.0,1.0,C,...,0.0,C,0.599539,0.0,1.0,2.55,0.77,2,1.130589,0.06849
4,4658151,dsgdb9nsd_000004,3,1,2JHC,H,1.661639,0.0,1.0,C,...,0.0,C,-0.599539,0.0,1.0,2.55,0.77,2,1.130589,0.06849
